In [10]:
from functools import partial

import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier

In [14]:
def objective(X, y, trial):
    """最小化する目的関数"""
    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'verbosity': -1,
        'boosting_type': trial.suggest_categorical('boosting', ['gbdt', 'dart', 'goss']),
        'num_leaves': trial.suggest_int('num_leaves', 10, 1000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-8, 1.0)
    }

    # モデルを作る
    model = LGBMClassifier(**params)

    # 5-Fold CV / Accuracy でモデルを評価する
    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_validate(model, X=X, y=y, cv=kf)
    # 最小化なので 1.0 からスコアを引く
    return 1.0 - scores['test_score'].mean()

In [2]:
import os
import pandas as pd

In [3]:
data = pd.read_csv("~/.kaggle/competitions/titanic/train.csv")

In [4]:
X = data[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Cabin', 'Embarked']]
X = pd.get_dummies(X, columns=['Pclass', 'Sex', 'SibSp', 'Parch', 'Cabin', 'Embarked']).fillna(0)
y = data['Survived']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [11]:
model = LGBMClassifier()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))

/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.8156424581005587


In [15]:
# 目的関数にデータを適用する
f = partial(objective, X_train, y_train)

# 最適化のセッションを作る
study = optuna.create_study()

# 100 回試行する
study.optimize(f, n_trials=100)

# 最適化したパラメータを出力する
print('params:', study.best_params)

/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/preprocess

/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/preprocess

/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/preprocess

[I 2019-05-11 18:36:37,354] Finished trial#3 resulted in value: 0.29487927565392347. Current best value is 0.18823887771070869 with parameters: {'boosting': 'dart', 'num_leaves': 641, 'learning_rate': 0.010656002586766487}.
/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambig

/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/preprocess

...

[I 2019-05-11 18:41:13,978] Finished trial#95 resulted in value: 0.37639839034205236. Current best value is 0.18246534764140399 with parameters: {'boosting': 'dart', 'num_leaves': 177, 'learning_rate': 0.02804330071474566}.
/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambig

/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/preprocess

/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/preprocess

[I 2019-05-11 18:41:20,354] Finished trial#99 resulted in value: 0.19105633802816901. Current best value is 0.18246534764140399 with parameters: {'boosting': 'dart', 'num_leaves': 177, 'learning_rate': 0.02804330071474566}.


params: {'boosting': 'dart', 'num_leaves': 177, 'learning_rate': 0.02804330071474566}


In [16]:
op_model = LGBMClassifier(**study.best_params)
op_model.fit(X_train, y_train)
print(op_model.score(X_test, y_test))

/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.7988826815642458
